# Learn Neo4j and Cypher in Notebook

### References

- https://github.com/elena/py2neo-quickstart
- Cypher Fundamentals by Dave

### Install Neo4j Desktop

```
$ cd ~/neo4j
$ ./neo4j-desktop-1.4.1-x86_64.AppImage
```

Open `neo4j` browser, type:
`:play movies`

Run the script to create `Movies` graph database

### [Cypher Styles](https://github.com/opencypher/openCypher/blob/master/docs/style-guide.adoc)
- Case Sensitive
    - Node labels
    - Relationship types
    - Property keys
- Case Insensitive
    - Cypher keywords
    
- Cypher keywords and clauses should be written in all caps and on their own line
- Functions should be written in lower camel case.

In [1]:
from py2neo import Graph,Node,Relationship

In [15]:
import pandas as pd
pd.set_option('display.max_colwidth', None)   # column value displayed in full

In [2]:
from _connstr import CONN_STR

In [3]:
url, username, password = CONN_STR["url"],CONN_STR["user"],CONN_STR["password"]
graph = Graph(url, auth=(username, password))

### Cypher - Intro

find 10 persons

In [6]:
query = """
MATCH (p:Person)
RETURN p
LIMIT 10
"""

results = graph.run(query)
type(results)

py2neo.database.work.Cursor

In [7]:
# results is serialized to dataframe
df = results.to_data_frame()
df

,p
0,"{'born': 1964, 'name': 'Keanu Reeves'}"
1,"{'born': 1967, 'name': 'Carrie-Anne Moss'}"
2,"{'born': 1961, 'name': 'Laurence Fishburne'}"
3,"{'born': 1960, 'name': 'Hugo Weaving'}"
4,"{'born': 1967, 'name': 'Lilly Wachowski'}"
5,"{'born': 1965, 'name': 'Lana Wachowski'}"
6,"{'born': 1952, 'name': 'Joel Silver'}"
7,"{'born': 1975, 'name': 'Charlize Theron'}"
8,"{'born': 1940, 'name': 'Al Pacino'}"
9,"{'born': 1944, 'name': 'Taylor Hackford'}"


In [8]:
query = """
MATCH (p:Person)
RETURN p
LIMIT 10
"""

results = graph.run(query)

# results is serialized to numpy array
results.to_ndarray()

array([[Node('Person', born=1964, name='Keanu Reeves')],
       [Node('Person', born=1967, name='Carrie-Anne Moss')],
       [Node('Person', born=1961, name='Laurence Fishburne')],
       [Node('Person', born=1960, name='Hugo Weaving')],
       [Node('Person', born=1967, name='Lilly Wachowski')],
       [Node('Person', born=1965, name='Lana Wachowski')],
       [Node('Person', born=1952, name='Joel Silver')],
       [Node('Person', born=1975, name='Charlize Theron')],
       [Node('Person', born=1940, name='Al Pacino')],
       [Node('Person', born=1944, name='Taylor Hackford')]], dtype=object)

In [9]:
query = """
MATCH (p:Person)
RETURN p
LIMIT 10
"""

results = graph.run(query)

# must re-run query
results.to_table()

p
"(_1:Person {born: 1964, name: 'Keanu Reeves'})"
"(_2:Person {born: 1967, name: 'Carrie-Anne Moss'})"
"(_3:Person {born: 1961, name: 'Laurence Fishburne'})"
"(_4:Person {born: 1960, name: 'Hugo Weaving'})"
"(_5:Person {born: 1967, name: 'Lilly Wachowski'})"
"(_6:Person {born: 1965, name: 'Lana Wachowski'})"
"(_7:Person {born: 1952, name: 'Joel Silver'})"
"(_12:Person {born: 1975, name: 'Charlize Theron'})"
"(_13:Person {born: 1940, name: 'Al Pacino'})"
"(_14:Person {born: 1944, name: 'Taylor Hackford'})"


In [10]:
query = """
// read 10 persons
MATCH (p:Person)
RETURN "Person" as Type, p.name as Name, p.born as Birth_Year
LIMIT 10;
"""

df = graph.run(query).to_data_frame()
df.head(5)

,Type,Name,Birth_Year
0,Person,Keanu Reeves,1964
1,Person,Carrie-Anne Moss,1967
2,Person,Laurence Fishburne,1961
3,Person,Hugo Weaving,1960
4,Person,Lilly Wachowski,1967


### Cypher - MATCH (Read/Query

Find 10 actors with their roles played in the movie

In [14]:
query = """
// find actors and their movies
MATCH (p:Person)-[r:ACTED_IN]->(m:Movie)
RETURN p, r, m
LIMIT 10;
"""

df = graph.run(query).to_data_frame()
df.head(5)

,p,r,m
0,"{'born': 1960, 'name': 'Hugo Weaving'}",{'roles': ['Agent Smith']},"{'tagline': 'Welcome to the Real World', 'title': 'The Matrix', 'released': 1999}"
1,"{'born': 1961, 'name': 'Laurence Fishburne'}",{'roles': ['Morpheus']},"{'tagline': 'Welcome to the Real World', 'title': 'The Matrix', 'released': 1999}"
2,"{'born': 1967, 'name': 'Carrie-Anne Moss'}",{'roles': ['Trinity']},"{'tagline': 'Welcome to the Real World', 'title': 'The Matrix', 'released': 1999}"
3,"{'born': 1964, 'name': 'Keanu Reeves'}",{'roles': ['Neo']},"{'tagline': 'Welcome to the Real World', 'title': 'The Matrix', 'released': 1999}"
4,"{'born': 1960, 'name': 'Hugo Weaving'}",{'roles': ['Agent Smith']},"{'tagline': 'Free your mind', 'title': 'The Matrix Reloaded', 'released': 2003}"


In [50]:
df.iloc[0]

p             {'born': 1967, 'name': 'James Marshall'}
r                    {'roles': ['Pfc. Louden Downey']}
m    {'tagline': 'In the heart of the nation's capi...
Name: 0, dtype: object

In [52]:
df.iloc[0].p["name"], df.iloc[0].r["roles"], df.iloc[0].m["title"]  

('James Marshall', ['Pfc. Louden Downey'], 'A Few Good Men')

In [55]:
query = """
// variable
MATCH p = (:Person)-[:ACTED_IN]->(:Movie)
RETURN p
LIMIT 2;
"""

graph.run(query).data()

[{'p': Path(Node('Person', born=1967, name='James Marshall'), ACTED_IN(Node('Person', born=1967, name='James Marshall'), Node('Movie', released=1992, tagline="In the heart of the nation's capital, in a courthouse of the U.S. government, one man will stop at nothing to keep his honor, and one will stop at nothing to find the truth.", title='A Few Good Men')))},
 {'p': Path(Node('Person', born=1943, name='J.T. Walsh'), ACTED_IN(Node('Person', born=1943, name='J.T. Walsh'), Node('Movie', released=1992, tagline="In the heart of the nation's capital, in a courthouse of the U.S. government, one man will stop at nothing to keep his honor, and one will stop at nothing to find the truth.", title='A Few Good Men')))}]

In [65]:
query = """
//WHERE
MATCH (p:Person)
WHERE p.name = 'Tom Hanks'
RETURN p;
"""

graph.run(query).data()

[{'p': Node('Person', born=1956, name='Tom Hanks')}]

In [66]:
query = """
// WHERE with pattern
MATCH (p:Person)-[:WROTE]->(m:Movie)
WHERE (p)-[:PRODUCED]->(m)
RETURN p.name, m.title
LIMIT 3;
"""

graph.run(query).data()

[{'p.name': 'Cameron Crowe', 'm.title': 'Jerry Maguire'},
 {'p.name': 'Nora Ephron', 'm.title': 'When Harry Met Sally'},
 {'p.name': 'Lana Wachowski', 'm.title': 'V for Vendetta'}]

In [67]:
query = """
// WHERE with pattern
MATCH (p:Person)-[:WROTE]->(m:Movie)
WHERE NOT (p)-[:PRODUCED]->(m)
RETURN p.name, m.title
LIMIT 3;
"""

graph.run(query).data()

[{'p.name': 'Aaron Sorkin', 'm.title': 'A Few Good Men'},
 {'p.name': 'Jim Cash', 'm.title': 'Top Gun'},
 {'p.name': 'David Mitchell', 'm.title': 'Cloud Atlas'}]

In [68]:
query = """
MATCH (p:Person)
WHERE NOT (p)-[:ACTED_IN|:DIRECTED]->(:Movie)
RETURN p
LIMIT 3;
"""

graph.run(query).data()

[{'p': Node('Person', born=1952, name='Joel Silver')},
 {'p': Node('Person', born=1941, name='Jim Cash')},
 {'p': Node('Person', born=1969, name='David Mitchell')}]

In [73]:
query = """
MATCH (m:Movie)
WHERE m.title CONTAINS 'Matrix'
RETURN m.title
LIMIT 3;
"""

graph.run(query).data()

[{'m.title': 'The Matrix'},
 {'m.title': 'The Matrix Reloaded'},
 {'m.title': 'The Matrix Revolutions'}]

In [70]:
query = """
// string functions
MATCH (m:Movie)
WHERE m.title STARTS WITH 'The Matrix'
RETURN m.title
LIMIT 4;
"""

graph.run(query).data()

[{'m.title': 'The Matrix'},
 {'m.title': 'The Matrix Reloaded'},
 {'m.title': 'The Matrix Revolutions'}]

In [74]:
query = """
// string functions
MATCH (m:Movie)
WHERE m.title ENDS WITH 'Seattle'
RETURN m.title
LIMIT 4;
"""

graph.run(query).data()

[{'m.title': 'Sleepless in Seattle'}]

In [72]:
query = """
MATCH (p:Person)-[:ACTED_IN]->(m:Movie)
WHERE p.name STARTS WITH 'Tom'
RETURN p, m
LIMIT 4;
"""

graph.run(query).data()

[{'p': Node('Person', born=1962, name='Tom Cruise'),
  'm': Node('Movie', released=2000, tagline='The rest of his life begins now.', title='Jerry Maguire')},
 {'p': Node('Person', born=1962, name='Tom Cruise'),
  'm': Node('Movie', released=1986, tagline='I feel the need, the need for speed.', title='Top Gun')},
 {'p': Node('Person', born=1962, name='Tom Cruise'),
  'm': Node('Movie', released=1992, tagline="In the heart of the nation's capital, in a courthouse of the U.S. government, one man will stop at nothing to keep his honor, and one will stop at nothing to find the truth.", title='A Few Good Men')},
 {'p': Node('Person', born=1933, name='Tom Skerritt'),
  'm': Node('Movie', released=1986, tagline='I feel the need, the need for speed.', title='Top Gun')}]

In [77]:
query = """
// regex
MATCH (m:Movie)
WHERE m.title =~ '.*[0-9]+.*'
RETURN m.title
LIMIT 3;
"""

graph.run(query).data()

[{'m.title': 'Apollo 13'}]

In [78]:
query = """
// aggregate, order by, limit
MATCH (p:Person)-[:ACTED_IN]->(m:Movie)
RETURN p.name, count(*) AS movies
ORDER BY movies DESC
LIMIT 5;
"""

graph.run(query).data()

[{'p.name': 'Tom Hanks', 'movies': 12},
 {'p.name': 'Keanu Reeves', 'movies': 7},
 {'p.name': 'Jack Nicholson', 'movies': 5},
 {'p.name': 'Hugo Weaving', 'movies': 5},
 {'p.name': 'Meg Ryan', 'movies': 5}]

In [79]:
query = """
// multiple match
MATCH (p:Person)-[:ACTED_IN]->(m:Movie),
      (other:Person)-[:ACTED_IN]->(m)
WHERE p.name = 'Meg Ryan'
RETURN other.name, m.title
ORDER BY other.name
LIMIT 3;
"""

graph.run(query).data()

[{'other.name': 'Anthony Edwards', 'm.title': 'Top Gun'},
 {'other.name': 'Bill Pullman', 'm.title': 'Sleepless in Seattle'},
 {'other.name': 'Billy Crystal', 'm.title': 'When Harry Met Sally'}]

In [80]:
query = """
// multiple match
MATCH (p:Person)-[:ACTED_IN]->(m:Movie),
      (other:Person)-[:ACTED_IN]->(m),
      (director:Person)-[:DIRECTED]->(m)
WHERE p.name = 'Meg Ryan'
RETURN m.title AS movie, 
       director.name AS director,
       other.name AS coActor
LIMIT 3;
"""

graph.run(query).data()

[{'movie': 'Sleepless in Seattle',
  'director': 'Nora Ephron',
  'coActor': 'Victor Garber'},
 {'movie': 'Sleepless in Seattle',
  'director': 'Nora Ephron',
  'coActor': 'Tom Hanks'},
 {'movie': 'Sleepless in Seattle',
  'director': 'Nora Ephron',
  'coActor': 'Bill Pullman'}]

In [81]:
query = """
// OPTINOAL MATCH
MATCH (p:Person)
WHERE p.name STARTS WITH 'Tom'
OPTIONAL MATCH (p)-[:DIRECTED]->(m:Movie)
RETURN p.name, m.title
LIMIT 5;
"""

graph.run(query).data()

[{'p.name': 'Tom Cruise', 'm.title': None},
 {'p.name': 'Tom Skerritt', 'm.title': None},
 {'p.name': 'Tom Hanks', 'm.title': 'That Thing You Do'},
 {'p.name': 'Tom Tykwer', 'm.title': 'Cloud Atlas'}]

Find actors with more than 5 movies

In [16]:
query = """
// WITH Clause
MATCH (p:Person)-[:ACTED_IN]->(m:Movie)
WITH p, collect(m.title) AS movies
WHERE size(movies) > 5
RETURN p.name, movies;
"""

graph.run(query).data()

[{'p.name': 'Keanu Reeves',
  'movies': ['The Matrix',
   'The Matrix Reloaded',
   'The Matrix Revolutions',
   "The Devil's Advocate",
   'The Replacements',
   'Johnny Mnemonic',
   "Something's Gotta Give"]},
 {'p.name': 'Tom Hanks',
  'movies': ["You've Got Mail",
   'Sleepless in Seattle',
   'Joe Versus the Volcano',
   'That Thing You Do',
   'Cloud Atlas',
   'The Da Vinci Code',
   'The Green Mile',
   'Apollo 13',
   'Cast Away',
   "Charlie Wilson's War",
   'The Polar Express',
   'A League of Their Own']}]

In [18]:
query = """
// WITH Clause
MATCH (p:Person)-[:ACTED_IN]->(m:Movie)
WITH p, m
ORDER BY m.title
WITH p, collect(m.title) AS movies
WHERE size(movies) > 5
RETURN p.name as Name, movies;
"""

graph.run(query).to_table()

Name,movies
Tom Hanks,"['A League of Their Own', 'Apollo 13', 'Cast Away', ""Charlie Wilson's War"", 'Cloud Atlas', 'Joe Versus the Volcano', 'Sleepless in Seattle', 'That Thing You Do', 'The Da Vinci Code', 'The Green Mile', 'The Polar Express', ""You've Got Mail""]"
Keanu Reeves,"['Johnny Mnemonic', ""Something's Gotta Give"", ""The Devil's Advocate"", 'The Matrix', 'The Matrix Reloaded', 'The Matrix Revolutions', 'The Replacements']"


Get the two oldest people and their three most recent movies.

In [23]:
query = """
// Get the two oldest people and their three most recent movies.
MATCH (actor:Person) 
WITH actor 
ORDER BY actor.born 
LIMIT 2 
MATCH (actor)-[:ACTED_IN]->(movie:Movie) 
WITH actor, movie 
ORDER BY movie.released DESC 
RETURN actor.name, 
       2016 - actor.born AS age, 
       collect(movie.title)[..3] AS movies;
"""

graph.run(query).to_table()

actor.name,age,movies
Gene Hackman,86,"['The Replacements', 'The Birdcage', 'Unforgiven']"
Max von Sydow,87,"['Snow Falling on Cedars', 'What Dreams May Come']"


In [27]:
query = """
// Get the two oldest people and their three most recent movies.
MATCH (actor:Person) 
WITH actor 
ORDER BY actor.born 
LIMIT 2 
MATCH (actor)-[:ACTED_IN]->(movie:Movie) 
WITH actor, movie 
ORDER BY movie.released DESC 
RETURN actor.name, 
       2016 - actor.born AS age,
       movie.released,
       movie.title
ORDER BY actor.name;
"""

graph.run(query).to_table()

actor.name,age,movie.released,movie.title
Gene Hackman,86,2000,The Replacements
Gene Hackman,86,1996,The Birdcage
Gene Hackman,86,1992,Unforgiven
Max von Sydow,87,1999,Snow Falling on Cedars
Max von Sydow,87,1998,What Dreams May Come


In [36]:
query = """
// distinct
MATCH (p:Person)-[:ACTED_IN]->(m:Movie),
      (other:Person)-[:ACTED_IN]->(m)
WHERE p.name = 'Meg Ryan'
RETURN collect(DISTINCT other.name) as co_stars
;
"""

graph.run(query).to_table()

co_stars
"['Victor Garber', 'Tom Hanks', 'Bill Pullman', 'Rita Wilson', ""Rosie O'Donnell"", 'Parker Posey', 'Greg Kinnear', 'Steve Zahn', 'Dave Chappelle', 'Val Kilmer', 'Tom Skerritt', 'Kelly McGillis', 'Tom Cruise', 'Anthony Edwards', 'Carrie Fisher', 'Billy Crystal', 'Bruno Kirby', 'Nathan Lane']"


In [37]:
query = """
// collect to array
MATCH (p:Person)-[:ACTED_IN]->(m:Movie)
WHERE m.title STARTS WITH 'The Matrix'
RETURN m.title, collect(p.name) AS actors;
"""

graph.run(query).to_table()

m.title,actors
The Matrix,"['Hugo Weaving', 'Laurence Fishburne', 'Carrie-Anne Moss', 'Keanu Reeves']"
The Matrix Reloaded,"['Hugo Weaving', 'Laurence Fishburne', 'Carrie-Anne Moss', 'Keanu Reeves']"
The Matrix Revolutions,"['Hugo Weaving', 'Laurence Fishburne', 'Carrie-Anne Moss', 'Keanu Reeves']"


Get top producers

In [38]:
query = """
// size
MATCH (p:Person)
RETURN p.name, 
       size((p)-[:PRODUCED]->(:Movie)) AS prod
ORDER BY prod DESC
LIMIT 5;
"""

graph.run(query).to_table()

p.name,prod
Joel Silver,6
Lilly Wachowski,2
Lana Wachowski,2
Rob Reiner,1
Cameron Crowe,1


Find producers with more than 5 movie titles

In [40]:
query = """
MATCH (p:Person)
WHERE size((p)-[:PRODUCED]->(:Movie)) > 5
RETURN p.name;
"""

graph.run(query).to_table()

p.name
Joel Silver


In [49]:
query = """
// Find the top five actors by how many movies they’ve acted in and movies they’ve directed, if any
MATCH (p:Person)
WITH p, size((p)-[:ACTED_IN]->(:Movie)) AS movies
ORDER BY movies DESC
LIMIT 5
OPTIONAL MATCH (p)-[:ACTED_IN]->(m:Movie)
OPTIONAL MATCH (p)-[:DIRECTED]->(m2:Movie)
RETURN p.name, movies, collect(m.title), m2.title;
"""

graph.run(query).to_table()

p.name,movies,collect(m.title),m2.title
Tom Hanks,12,"['Apollo 13', ""You've Got Mail"", 'A League of Their Own', 'Joe Versus the Volcano', 'That Thing You Do', 'The Da Vinci Code', 'Cloud Atlas', 'Cast Away', 'The Green Mile', 'Sleepless in Seattle', 'The Polar Express', ""Charlie Wilson's War""]",That Thing You Do
Keanu Reeves,7,"[""Something's Gotta Give"", 'The Replacements', 'Johnny Mnemonic', ""The Devil's Advocate"", 'The Matrix Revolutions', 'The Matrix Reloaded', 'The Matrix']",null
Meg Ryan,5,"['Sleepless in Seattle', ""You've Got Mail"", 'Top Gun', 'When Harry Met Sally', 'Joe Versus the Volcano']",null
Hugo Weaving,5,"['Cloud Atlas', 'V for Vendetta', 'The Matrix Revolutions', 'The Matrix Reloaded', 'The Matrix']",null
Jack Nicholson,5,"['As Good as It Gets', ""One Flew Over the Cuckoo's Nest"", ""Something's Gotta Give"", 'Hoffa', 'A Few Good Men']",null


#### UNWIND: list into row

In [50]:
query = """
WITH [1, 2, 3] AS list
RETURN list;
"""

graph.run(query).to_table()

list
"[1, 2, 3]"


In [53]:
query = """
// list to row
WITH [1, 2, 3] AS list
UNWIND list AS row
RETURN row;
"""

graph.run(query).to_table()

row
1
2
3


In [57]:
query = """
MATCH (p:Person)-[:ACTED_IN]->(m:Movie)
WITH p, m
ORDER BY m.released
WITH p, collect(m)[..3] AS topThree
UNWIND topThree AS m
MATCH (dir:Person)-[:DIRECTED]->(m)
RETURN p.name, m.title, dir.name
LIMIT 7;
"""

graph.run(query).to_table()

p.name,m.title,dir.name
Danny DeVito,One Flew Over the Cuckoo's Nest,Milos Forman
Danny DeVito,Hoffa,Danny DeVito
Jack Nicholson,One Flew Over the Cuckoo's Nest,Milos Forman
Jack Nicholson,A Few Good Men,Rob Reiner
Jack Nicholson,Hoffa,Danny DeVito
Val Kilmer,Top Gun,Tony Scott
Meg Ryan,Top Gun,Tony Scott


In [58]:
query = """
// Variable length paths between two persons
MATCH p = (p1:Person)-[:ACTED_IN*..4]-(p2:Person)
WHERE p1.name = 'Tom Cruise' AND      
      p2.name = 'Kevin Bacon'
RETURN p;
"""

graph.run(query).to_table()

p
(Tom Cruise)-[:ACTED_IN {}]->(_37)<-[:ACTED_IN {}]-(Cuba Gooding Jr.)-[:ACTED_IN {}]->(_15)<-[:ACTED_IN {}]-(Kevin Bacon)
(Tom Cruise)-[:ACTED_IN {}]->(_15)<-[:ACTED_IN {}]-(Kevin Bacon)


In [59]:
query = """
MATCH p = (p1:Person)-[:ACTED_IN*4..6]-(p2:Person)
WHERE p1.name = 'Tom Hanks' AND      
      p2.name = 'Kevin Bacon'
RETURN p;
"""

graph.run(query).to_table()

p
(Tom Hanks)-[:ACTED_IN {}]->(_144)<-[:ACTED_IN {}]-(Gary Sinise)-[:ACTED_IN {}]->(_130)<-[:ACTED_IN {}]-(Sam Rockwell)-[:ACTED_IN {}]->(_137)<-[:ACTED_IN {}]-(Kevin Bacon)
(Tom Hanks)-[:ACTED_IN {}]->(_67)<-[:ACTED_IN {}]-(Greg Kinnear)-[:ACTED_IN {}]->(_52)<-[:ACTED_IN {}]-(Jack Nicholson)-[:ACTED_IN {}]->(_15)<-[:ACTED_IN {}]-(Kevin Bacon)
(Tom Hanks)-[:ACTED_IN {}]->(_67)<-[:ACTED_IN {}]-(Greg Kinnear)-[:ACTED_IN {}]->(_52)<-[:ACTED_IN {}]-(Cuba Gooding Jr.)-[:ACTED_IN {}]->(_15)<-[:ACTED_IN {}]-(Kevin Bacon)
(Tom Hanks)-[:ACTED_IN {}]->(_67)<-[:ACTED_IN {}]-(Meg Ryan)-[:ACTED_IN {}]->(_73)<-[:ACTED_IN {}]-(Tom Hanks)-[:ACTED_IN {}]->(_144)<-[:ACTED_IN {}]-(Kevin Bacon)
(Tom Hanks)-[:ACTED_IN {}]->(_67)<-[:ACTED_IN {}]-(Meg Ryan)-[:ACTED_IN {}]->(_29)<-[:ACTED_IN {}]-(Tom Cruise)-[:ACTED_IN {}]->(_15)<-[:ACTED_IN {}]-(Kevin Bacon)
(Tom Hanks)-[:ACTED_IN {}]->(_67)<-[:ACTED_IN {}]-(Meg Ryan)-[:ACTED_IN {}]->(_78)<-[:ACTED_IN {}]-(Tom Hanks)-[:ACTED_IN {}]->(_144)<-[:ACTED_IN {}]-(Kevin Bacon)
(Tom Hanks)-[:ACTED_IN {}]->(_162)<-[:ACTED_IN {}]-(Rosie O'Donnell)-[:ACTED_IN {}]->(_73)<-[:ACTED_IN {}]-(Tom Hanks)-[:ACTED_IN {}]->(_144)<-[:ACTED_IN {}]-(Kevin Bacon)
(Tom Hanks)-[:ACTED_IN {}]->(_162)<-[:ACTED_IN {}]-(Bill Paxton)-[:ACTED_IN {}]->(_144)<-[:ACTED_IN {}]-(Kevin Bacon)
(Tom Hanks)-[:ACTED_IN {}]->(_78)<-[:ACTED_IN {}]-(Meg Ryan)-[:ACTED_IN {}]->(_73)<-[:ACTED_IN {}]-(Tom Hanks)-[:ACTED_IN {}]->(_144)<-[:ACTED_IN {}]-(Kevin Bacon)
(Tom Hanks)-[:ACTED_IN {}]->(_78)<-[:ACTED_IN {}]-(Meg Ryan)-[:ACTED_IN {}]->(_67)<-[:ACTED_IN {}]-(Tom Hanks)-[:ACTED_IN {}]->(_144)<-[:ACTED_IN {}]-(Kevin Bacon)


In [66]:
query = """
MATCH p = (p1:Person)-[:ACTED_IN*4..6]-(p2:Person)
WHERE p1.name = 'Tom Cruise' AND            
      p2.name = 'Kevin Bacon'
RETURN [x IN nodes(p) |  
  CASE WHEN x:Person THEN '(p) '+x.name       
       WHEN x:Movie  THEN '(m) '+x.title  
  ELSE '' END
];
"""

graph.run(query).to_table()

[x IN nodes(p) | CASE WHEN x:Person THEN '(p) '+x.name WHEN x:Movie THEN '(m) '+x.title ELSE '' END ]
"['(p) Tom Cruise', '(m) Jerry Maguire', ""(p) Jerry O'Connell"", '(m) Stand By Me', '(p) Kiefer Sutherland', '(m) A Few Good Men', '(p) Kevin Bacon']"
"['(p) Tom Cruise', '(m) Jerry Maguire', '(p) Bonnie Hunt', '(m) The Green Mile', '(p) Sam Rockwell', '(m) Frost/Nixon', '(p) Kevin Bacon']"
"['(p) Tom Cruise', '(m) Jerry Maguire', '(p) Bonnie Hunt', '(m) The Green Mile', '(p) Tom Hanks', '(m) Apollo 13', '(p) Kevin Bacon']"
"['(p) Tom Cruise', '(m) Jerry Maguire', '(p) Bonnie Hunt', '(m) The Green Mile', '(p) Gary Sinise', '(m) Apollo 13', '(p) Kevin Bacon']"
"['(p) Tom Cruise', '(m) Jerry Maguire', '(p) Cuba Gooding Jr.', '(m) As Good as It Gets', '(p) Jack Nicholson', '(m) A Few Good Men', '(p) Kevin Bacon']"
"['(p) Tom Cruise', '(m) Jerry Maguire', '(p) Cuba Gooding Jr.', '(m) A Few Good Men', '(p) Kevin Bacon']"
"['(p) Tom Cruise', '(m) Top Gun', '(p) Meg Ryan', '(m) Sleepless in Seattle', '(p) Tom Hanks', '(m) Apollo 13', '(p) Kevin Bacon']"
"['(p) Tom Cruise', '(m) Top Gun', '(p) Meg Ryan', ""(m) You've Got Mail"", '(p) Tom Hanks', '(m) Apollo 13', '(p) Kevin Bacon']"
"['(p) Tom Cruise', '(m) Top Gun', '(p) Meg Ryan', '(m) Joe Versus the Volcano', '(p) Tom Hanks', '(m) Apollo 13', '(p) Kevin Bacon']"
"['(p) Tom Cruise', '(m) A Few Good Men', '(p) J.T. Walsh', '(m) Hoffa', '(p) Jack Nicholson', '(m) A Few Good Men', '(p) Kevin Bacon']"


#### Shortest path

In [67]:
query = """
// shortest path
MATCH p = shortestPath((p1:Person)-[*]-(p2:Person))
WHERE p1.name = 'Tom Cruise' AND      
      p2.name = 'Kevin Bacon'
RETURN p;
"""

graph.run(query).to_table()

p
(Tom Cruise)-[:ACTED_IN {}]->(_15)<-[:ACTED_IN {}]-(Kevin Bacon)


In [68]:
query = """
// shortest path between two movies via any rel type
MATCH p = shortestPath((m1:Movie)-[*]-(m2:Movie))
WHERE m1.title = 'Top Gun' AND      
      m2.title = 'The Matrix'
RETURN p;
"""

graph.run(query).to_table()

p
(_29)<-[:ACTED_IN {}]-(Tom Cruise)-[:ACTED_IN {}]->(_15)<-[:ACTED_IN {}]-(James Marshall)-[:DIRECTED {}]->(_116)<-[:PRODUCED {}]-(Joel Silver)-[:PRODUCED {}]->(_0)


In [70]:
query = """
// Bacon_Number
MATCH p = shortestPath((c:Person)-[:KNOWS*]->(bacon:Person))
WHERE c.name = 'Tom Hanks' AND      
      bacon.name = 'Kevin Bacon'
RETURN length(p) AS Bacon_Number, [n IN nodes(p) | n.name] AS Names;
"""

graph.run(query).to_table()

In [71]:
query = """
// Recommend: 3 actors that Keanu Reeves should work with, but hasn’t
MATCH (p:Person)-[:ACTED_IN]->()<-[:ACTED_IN]-(c),
(c)-[:ACTED_IN]->()<-[:ACTED_IN]-(coc)
WHERE p.name = 'Keanu Reeves' 
	AND coc <> p
 	AND NOT (p)-[:ACTED_IN]->()<-[:ACTED_IN]-(coc) 
RETURN coc.name as Colleague_of_colleague, count(coc) as 
Weight
ORDER BY Weight DESC
LIMIT 3;
"""

graph.run(query).to_table()

Colleague_of_colleague,Weight
Tom Hanks,4
Jim Broadbent,3
Halle Berry,3


### CREATE & MERGE (Write) - CU

In [85]:
query = """
// CREATE (INSERT)
CREATE (m:Movie {title:'Mystic River', released:2003})
RETURN m;
"""

graph.run(query).to_table()

m
"(_174:Movie {released: 2003, title: 'Mystic River'})"


In [86]:
query = """
// CREATE Rel
MATCH (m:Movie {title: 'Mystic River'})
MATCH (p:Person {name: 'Kevin Bacon'})
CREATE (p)-[r:ACTED_IN {roles: ['Sean']}]->(m)
RETURN p, r, m;
"""

graph.run(query).to_table()

p,r,m
"(_19:Person {born: 1958, name: 'Kevin Bacon'})",(Kevin Bacon)-[:ACTED_IN {roles: ['Sean']}]->(_174),"(_174:Movie {released: 2003, title: 'Mystic River'})"


In [87]:
query = """
// SET (UPDATE)
MATCH (m:Movie {title: 'Mystic River'})
SET m.tagline = 'We bury our sins here, Dave. We wash them clean.' 
RETURN m;
"""

graph.run(query).to_table()

m
"(_174:Movie {released: 2003, tagline: 'We bury our sins here, Dave. We wash them clean.', title: 'Mystic River'})"


In [88]:
query = """
// MERGE
MERGE (p:Person {name: 'Tom Hanks'})
RETURN p;
"""

graph.run(query).to_table()

p
"(_71:Person {born: 1956, name: 'Tom Hanks'})"


In [89]:
query = """
// Update node property
MERGE (p:Person {name: 'Tom Hanks', oscar: true})
RETURN p;
"""

graph.run(query).to_table()

p
"(_175:Person {name: 'Tom Hanks', oscar: true})"


In [90]:
query = """
MERGE (p:Person {name: 'Tom Hanks'})
SET p.oscar = false
RETURN p;
"""

graph.run(query).to_table()

p
"(_71:Person {born: 1956, name: 'Tom Hanks', oscar: false})"
"(_175:Person {name: 'Tom Hanks', oscar: false})"


In [91]:
query = """
// Create a new movie and rel
MERGE (p:Person {name: 'Tom Hanks'})-[:ACTED_IN]->(m:Movie {title: 'The Terminal'})
RETURN p, m;
"""

graph.run(query).to_table()

p,m
(_176:Person {name: 'Tom Hanks'}),(_177:Movie {title: 'The Terminal'})


In [92]:
query = """
MERGE (p:Person {name: 'Tom Hanks'})
MERGE (m:Movie {title: 'The Terminal'})
MERGE (p)-[r:ACTED_IN]->(m)
RETURN p, r, m;
"""

graph.run(query).to_table()

p,r,m
"(_71:Person {born: 1956, name: 'Tom Hanks', oscar: false})",(Tom Hanks)-[:ACTED_IN {}]->(_177),(_177:Movie {title: 'The Terminal'})
"(_175:Person {name: 'Tom Hanks', oscar: false})",(Tom Hanks)-[:ACTED_IN {}]->(_177),(_177:Movie {title: 'The Terminal'})
(_176:Person {name: 'Tom Hanks'}),(Tom Hanks)-[:ACTED_IN {}]->(_177),(_177:Movie {title: 'The Terminal'})


In [93]:
query = """
MERGE (p:Person {name: 'Tom Hanks'})
MERGE (m:Movie {title: 'The Terminal'})
MERGE (p)-[r:DIRECTED]->(m)
RETURN p, r, m;
"""

graph.run(query).to_table()

p,r,m
"(_71:Person {born: 1956, name: 'Tom Hanks', oscar: false})",(Tom Hanks)-[:DIRECTED {}]->(_177),(_177:Movie {title: 'The Terminal'})
"(_175:Person {name: 'Tom Hanks', oscar: false})",(Tom Hanks)-[:DIRECTED {}]->(_177),(_177:Movie {title: 'The Terminal'})
(_176:Person {name: 'Tom Hanks'}),(Tom Hanks)-[:DIRECTED {}]->(_177),(_177:Movie {title: 'The Terminal'})


In [102]:
query = """
MATCH (p:Person)
WHERE p.name = 'Your Name'
DELETE p;
"""

graph.run(query)

(No data)

Use `Ctrl-Enter` to execute below cell to see `updated` attribute changes

In [112]:
query = """
// MERGE ON CREATE / ON MATCH
MERGE (p:Person {name: 'Your Name'})
  ON CREATE SET p.created_at = timestamp(),
                p.updated_at = timestamp(),
                p.updated = 0 
  ON MATCH SET p.updated = p.updated + 1,
               p.updated_at = timestamp()
RETURN p.created_at, p.updated_at, p.updated;
"""

graph.run(query).to_table()

p.created_at,p.updated_at,p.updated
1615347851788,1615347923029,9


In [114]:
query = """
// Create KNOWS relationships between all actors who have worked together on the same movie
MATCH (a:Person)-[:ACTED_IN]->()<-[:ACTED_IN]-(b:Person)
MERGE (a)-[:KNOWS]-(b);
"""

graph.run(query)

(No data)

In [119]:
query = """
MATCH (a:Person)-[:KNOWS]-(b:Person)
RETURN a, b
LIMIT 3;
"""

graph.run(query).to_table()

a,b
"(_1:Person {born: 1964, name: 'Keanu Reeves'})","(_155:Person {born: 1946, name: 'Diane Keaton'})"
"(_1:Person {born: 1964, name: 'Keanu Reeves'})","(_3:Person {born: 1961, name: 'Laurence Fishburne'})"
"(_1:Person {born: 1964, name: 'Keanu Reeves'})","(_101:Person {born: 1947, name: 'Takeshi Kitano'})"


### CONSTRAINT and INDEX

### DELETE

In [81]:
query = """
// CREATE (INSERT)
CREATE (p:Person {name:'Emil Eifrem'})
RETURN p;
"""

graph.run(query)

 p                                   
-------------------------------------
 (_173:Person {name: 'Emil Eifrem'}) 

In [83]:
query = """
MATCH (p:Person)
WHERE p.name = 'Emil Eifrem'
DELETE p;
"""

graph.run(query)

(No data)

In [84]:
query = """
MATCH (p:Person)
WHERE p.name = 'Emil Eifrem'
RETURN p;
"""

graph.run(query).to_table()

### Cypher - DELETE

In [62]:
query = """
MATCH (p:Person)
WHERE p.name = 'Emil Eifrem'
RETURN p;
"""

graph.run(query)

 p                                             
-----------------------------------------------
 (_8:Person {born: 1978, name: 'Emil Eifrem'}) 

In [64]:
query = """
// delete one node and relationships
MATCH (p:Person)
WHERE p.name = 'Emil Eifrem'
DETACH DELETE p;
"""

graph.run(query)

(No data)

In [57]:
query = """
//Delete all Movie and Person nodes, and their relationships
MATCH (n) 
DETACH DELETE n;
"""

graph.run(query)

(No data)

In [60]:
query = """
//Prove that the Movie Graph is gone
MATCH (n) RETURN count(*);
"""

graph.run(query)

 count(*) 
----------
      171 

### Data Import